In [ ]:
!pip install pypdf2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 31.5 MB/s eta 0:00:00


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.6 MB/s eta 0:00:00


Load the dataset of job description and store it in dataset of job_profile with all features

```
# This is formatted as code
```



In [ ]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")
feature_datasets = {}
feature_names = dataset['train'].features.keys()
for feature_name in feature_names:
    feature_datasets[feature_name] = dataset['train'][feature_name]
job_profiles=pd.DataFrame(feature_datasets)
job_profiles
           # process_resume(resume_path)

,company_name,job_description,position_title,description_length,model_response
0,Google,minimum qualifications\nbachelors degree or eq...,Sales Specialist,2727,"{\n ""Core Responsibilities"": ""Responsible fo..."
1,Apple,description\nas an asc you will be highly infl...,Apple Solutions Consultant,828,"{\n ""Core Responsibilities"": ""as an asc you ..."
2,Netflix,its an amazing time to be joining netflix as w...,Licensing Coordinator - Consumer Products,3205,"{\n ""Core Responsibilities"": ""Help drive bus..."
3,Robert Half,description\n\nweb designers looking to expand...,Web Designer,2489,"{\n ""Core Responsibilities"": ""Designing webs..."
4,TrackFive,at trackfive weve got big goals were on a miss...,Web Developer,3167,"{\n ""Core Responsibilities"": ""Build and layo..."
...,...,...,...,...,...
848,Menards,job description\n\nparttime\n\nmake big money ...,Management Internship,1122,"{\n ""Core Responsibilities"": ""Responsibiliti..."
849,Parker,responsibilities\nparkers internship program w...,Human Resources Internship - Corporate (Year-...,3840,"{\n ""Core Responsibilities"": ""Assist in gene..."
850,Borgen Project,the borgen project is an innovative national ...,Writer / Journalist Internship,897,"{\n ""Core Responsibilities"": ""Write one arti..."
851,Wyndham Destinations,put the world on vacation\n\nat wyndham destin...,Inbound Customer Service / Sales (Remote),4604,"{\n ""Core Responsibilities"": ""Answer inbound..."


import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import PyPDF2
import re
import torch
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
from sklearn.metrics.pairwise import cosine_similarity



developed a pdf extractor here and stored extracted result in excel sheet. Dont need to un the code again and again. Just load the obtained excel sheet

In [ ]:

# #     print(item)
resume_info={}
def extract_text_from_pdf(pdf_file):
    text = ""
    with open(pdf_file, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
       # print(len(pdf_reader.pages))

        for page_num in range (len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            #print(page.extract_text)
            text += page.extract_text()

    return text

def extract_embeddings(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

    # Encode the text with DistilBERT
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract embeddings (CLS token)
    embeddings = outputs['last_hidden_state'][:, 0, :].numpy()
    return embeddings

# # You can now work with the extracted embeddings as needed
# print("Extracted embeddings shape:", embeddings.shape)
def extract_education_and_skills(text):
    education_details = []
    skills = []
    lines = text.split('\n')
    extracting_education = False
    extracting_skills = False

    for line in lines:
        # Check for the "Education" keyword
        if "Education" in line:
            extracting_education = True
            extracting_skills = False
            continue

        # Check for the "Skills" keyword
        if "Skills" in line:
            extracting_skills = True
            extracting_education = False
            continue

        # Extract education details
        if extracting_education:
            education_details.append(line.strip())

        # Extract skills
        if extracting_skills:
            skills.append(line.strip())

    return education_details, skills
def process_resume(resume_path):
     print(resume_path)
     if os.path.isfile(resume_path) and resume_path.endswith(".pdf"):
         # Extract text from the PDF resume
         pdf_text = extract_text_from_pdf(resume_path)

         # Extract embeddings using DistilBERT (for more advanced analysis)


         # Extract skills and education information
         education_info,skills_info= extract_education_and_skills(pdf_text)
         x=resume_path.split("/")
         # Extract category information (You need to define how you want to extract this)
         category = x[-2]
         #print(category)
         # Create a dictionary for the resume information
         resume_info[os.path.basename(resume_path)] = {
             "Category": category,
             "Skills": skills_info,
             "Education": education_info
         }

zip_file_path = "/content/archive.zip"

# Define the extraction directory
extraction_directory = "data"
with zipfile.ZipFile(zip_file_path, 'r') as zip:
      zip.extractall()
      print('Extracting all the files now...')
      print('Done!')
 # Locate the "data" folder within the extracted directory
data_folder = os.path.join(extraction_directory, "data")

 # Precompute and store resume embeddings
for category in os.listdir(data_folder):
     category_path = os.path.join(data_folder, category)

     if os.path.isdir(category_path):
         for resume_id in os.listdir(category_path):
             resume_path = os.path.join(category_path, resume_id)
             print(resume_path)
             process_resume(resume_path)
            # break    used to this break to get only first category extracted initally so that to see if correct results are obtained

#print(resume_info)
resume_df = pd.DataFrame.from_dict(resume_info, orient='index')

# Save the DataFrame to an Excel file
resume_excel_path = "resume_info.xlsx"
resume_df.to_excel(resume_excel_path)

here loaded the excel sheet obtanedd in pdf extractor

In [ ]:
loaded_resume_df=pd.read_excel('/content/resume_info.xlsx')
loaded_resume_df

extract_embeddings function

In [ ]:
def extract_embeddings(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs['last_hidden_state'][:, 0, :].numpy()
    return embeddings

find embeddings of model_response" columns of every job and store it in a list

In [ ]:
job_profile_embeddings = []

for _, job_row in job_profiles.iterrows():
      embeddings=extract_embeddings(job_row["model_response"],tokenizer,model)
      job_profile_embeddings.append(embeddings)
job_profile_embeddings

do siliar for each resume. Embeddings of skills and education combine is calculated.

In [ ]:
resume_embeddings = []

for _, resume_row in loaded_resume_df.iterrows():
    text_data = resume_row["Skills"] + " " + resume_row["Education"]
    resume_embeddings.append(extract_embeddings(text_data,tokenizer,model))
resume_embeddings



similarity_scores of each resume corrsponding to each job is stored

In [ ]:
similarity_scores=[]
for job_idx, job_profile_embedding in enumerate(job_profile_embeddings):
  for resume_idx, resume_embedding in enumerate(resume_embeddings):
     # print(loaded_resume_df.iloc[resume_idx,0])
      similarity = cosine_similarity(resume_embedding.reshape(1, -1), job_profile_embedding.reshape(1,-1))
      similarity_scores.append({
            "Resume_ID": loaded_resume_df.iloc[resume_idx,0],
            "Company": job_profiles.loc[job_idx, "company_name"],
            "Similarity": similarity
        })
similarity_df = pd.DataFrame(similarity_scores)




sorted according to similarity value in descending order

In [ ]:

similarity_df = similarity_df.sort_values(by="Similarity", ascending=False)
similarity_df

,Resume_ID,Company,Similarity
396887,26932091.pdf,Grant Thornton LLP (US),[[0.9647955]]
610652,44115326.pdf,Sherwin-Williams,[[0.96160114]]
397261,28383893.pdf,Grant Thornton LLP (US),[[0.9612926]]
1199593,28383893.pdf,DoorDash,[[0.9605603]]
873796,12820557.pdf,Fourth Floor,[[0.9604292]]
...,...,...,...
184034,36269672.pdf,Halliburton,[[0.73266727]]
186298,12632728.pdf,Halliburton,[[0.73266727]]
185811,17410700.pdf,Halliburton,[[0.73266727]]
185698,16911115.pdf,Halliburton,[[0.73266727]]


add top 5 resume to each job into top_matches_dict and concat all to obtain the final excel sheet

In [ ]:
top_matches_dict = {}
for job_profile_id, group in similarity_df.groupby("Company"):
    top_matches = group.head(5)
    top_matches_dict[job_profile_id]=top_matches
top_matches_df=pd.concat(top_matches_dict.values(), ignore_index=True)

In [ ]:
top_matches_df

,Resume_ID,Company,Similarity
0,10674770.pdf,Johnson Creek Development,[[0.94729215]]
1,16605640.pdf,Johnson Creek Development,[[0.9470488]]
2,27243670.pdf,Johnson Creek Development,[[0.9457487]]
3,27980446.pdf,Johnson Creek Development,[[0.9456996]]
4,11847784.pdf,Johnson Creek Development,[[0.9456408]]
...,...,...,...
4260,12820557.pdf,strategic HR inc.,[[0.9431532]]
4261,28383893.pdf,strategic HR inc.,[[0.94245064]]
4262,16121387.pdf,strategic HR inc.,[[0.93999445]]
4263,28745844.pdf,strategic HR inc.,[[0.93978643]]


Final excel sheet is obtained here

In [ ]:
top_matches_df.to_excel("top_matches_df.xlsx")